In [ ]:

import os
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-4.1")
from typing import Annotated
import csv
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

@tool
def get_deposits(customer_id: str = None) -> list:
    """Returns all bank deposits as a list of dicts. Optionally filter by customer_id."""
    print("get_deposits tool called with:", customer_id)
    with open("src/assets/Deposits.csv", newline="", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        deposits = list(reader)
    if customer_id:
        deposits = [d for d in deposits if d.get("CustomerID") == customer_id]
    return deposits

tool = TavilySearch(max_results=2)
tools = [tool,get_deposits]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [11]:
graph.invoke({"messages" : {"role" : "user", "content":"Tell me about the weather in Pensacola."}})

{'messages': [HumanMessage(content='Tell me about the weather in Pensacola.', additional_kwargs={}, response_metadata={}, id='43f86b71-37d1-4534-8637-701d96052ab1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_j6ESchxaD1MQHQmHkAdrHbET', 'function': {'arguments': '{"query":"current weather in Pensacola, FL"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 776, 'total_tokens': 798, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-BnWoCFOCPEXIlDrMcIsVJ2ME2BaIQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--79e62e3e-13ca-4aaf-9d27-f981de4165e0-0', tool_calls=[{'name': 'tavily_search',

In [ ]:
{'messages': [HumanMessage(content='Tell me about the weather in Pensacola.', additional_kwargs={}, response_metadata={}, id='43f86b71-37d1-4534-8637-701d96052ab1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_j6ESchxaD1MQHQmHkAdrHbET', 'function': {'arguments': '{"query":"current weather in Pensacola, FL"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 776, 'total_tokens': 798, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-BnWoCFOCPEXIlDrMcIsVJ2ME2BaIQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--79e62e3e-13ca-4aaf-9d27-f981de4165e0-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather in Pensacola, FL'}, 'id': 'call_j6ESchxaD1MQHQmHkAdrHbET', 'type': 'tool_call'}], usage_metadata={'input_tokens': 776, 'output_tokens': 22, 'total_tokens': 798, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  ToolMessage(content='{"query": "current weather in Pensacola, FL", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Pensacola, FL", "url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Pensacola\', \'region\': \'Florida\', \'country\': \'United States of America\', \'lat\': 30.4211, \'lon\': -87.2169, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1751143948, \'localtime\': \'2025-06-28 15:52\'}, \'current\': {\'last_updated_epoch\': 1751143500, \'last_updated\': \'2025-06-28 15:45\', \'temp_c\': 28.9, \'temp_f\': 84.0, \'is_day\': 1, \'condition\': {\'text\': \'Patchy light rain with thunder\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/386.png\', \'code\': 1273}, \'wind_mph\': 9.6, \'wind_kph\': 15.5, \'wind_degree\': 176, \'wind_dir\': \'S\', \'pressure_mb\': 1019.0, \'pressure_in\': 30.1, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 67, \'cloud\': 75, \'feelslike_c\': 32.6, \'feelslike_f\': 90.6, \'windchill_c\': 30.1, \'windchill_f\': 86.3, \'heatindex_c\': 35.6, \'heatindex_f\': 96.1, \'dewpoint_c\': 24.4, \'dewpoint_f\': 75.8, \'vis_km\': 16.0, \'vis_miles\': 9.0, \'uv\': 7.9, \'gust_mph\': 11.3, \'gust_kph\': 18.2}}", "score": 0.7599309, "raw_content": null}, {"title": "Weather in Pensacola in June 2025 (Florida) - detailed Weather Forecast ...", "url": "https://world-weather.info/forecast/usa/pensacola/june-2025/", "content": "World United States Florida Weather in Pensacola Weather in Pensacola in June 2025 Pensacola Weather Forecast for June 2025 is based on long term prognosis and previous years\' statistical data.", "score": 0.58948064, "raw_content": null}], "response_time": 2.16}', name='tavily_search', id='96fcd7f1-1db7-4298-af2d-e26362e6d459', tool_call_id='call_j6ESchxaD1MQHQmHkAdrHbET'),
  AIMessage(content='The current weather in Pensacola, FL is around 84°F (28.9°C) with patchy light rain and some thunder. It feels warmer, around 91°F (32.6°C), due to the humidity. Winds are coming from the south at about 9.6 mph. The sky is mostly cloudy and humidity is at 67%.\n\nIf you need a more detailed forecast or hourly updates, let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 1384, 'total_tokens': 1473, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-BnWoFiC3ovFJhIRxA8pxRl4I6Yllw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--aac6fa1f-2552-461e-80c8-3f6ef1637d75-0', usage_metadata={'input_tokens': 1384, 'output_tokens': 89, 'total_tokens': 1473, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}